In [1]:
import praw
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
reddit = praw.Reddit(client_id='Pj5o8QpNXXJY9A',
                     client_secret='pQKMRBmhp0In48NoNvvktfRo2eA',
                     password='prawisgreat',
                     user_agent='Reddit Unlocked CS196 Project @ UIUC',
                     username='RedditUnlocked196')

name = input('Please enter a subreddit name to be analyzed (without the "/r/"): ')
subreddit = reddit.subreddit(name)

print('Subreddit to be "unlocked" is: reddit.com/r/' + name)

Version 5.1.0 of praw is outdated. Version 5.2.0 was released Tuesday October 24, 2017.
Please enter a subreddit name to be analyzed (without the "/r/"): news
Subreddit to be "unlocked" is: reddit.com/r/news


In [3]:
threads_df = pd.DataFrame({
    'Title': (),
    'URL': (),
    'Upvote Ratio (%)': (),
    'Net Score': (),
    '# of Upvotes': (),
    '# of Downvotes': (),
    'Year Posted': (),
    'Month Posted': (),
    'Day Posted': (),
    'Self Post?': (),
    'Video Post?': ()
})

threads_df = threads_df[['Title', 'URL', 'Upvote Ratio (%)', 'Net Score', '# of Upvotes', '# of Downvotes', 
                         'Year Posted', 'Month Posted', 'Day Posted', 'Self Post?', 'Video Post?']]

In [4]:
# yearly top-threads collector

for thread in subreddit.top('year', limit = 10):
    actualUps = int((thread.upvote_ratio * thread.score)/(thread.upvote_ratio * 2 - 1))
    actualDowns = actualUps - thread.score
    gather = pd.Series([thread.title, thread.url, thread.upvote_ratio * 100, thread.score, 
                        actualUps, actualDowns, 
                        datetime.utcfromtimestamp(thread.created_utc).year, datetime.utcfromtimestamp(thread.created_utc).month,
                        datetime.utcfromtimestamp(thread.created_utc).day, thread.is_self, thread.is_video], 
                        index=['Title', 'URL', 'Upvote Ratio (%)', 'Net Score', '# of Upvotes', '# of Downvotes', 
                                  'Year Posted', 'Month Posted', 'Day Posted', 'Self Post?', 'Video Post?'])
    threads_df = threads_df.append(gather, ignore_index = True)
threads_df

,Title,URL,Upvote Ratio (%),Net Score,# of Upvotes,# of Downvotes,Year Posted,Month Posted,Day Posted,Self Post?,Video Post?
0,Chester Bennington of Linkin Park commits suicide,http://wtkr.com/2017/07/20/tmz-report-chester-...,84.0,123587.0,152666.0,29079.0,2017.0,7.0,20.0,0.0,0.0
1,Japanese firm gives non-smokers extra six days...,http://www.independent.co.uk/news/world/asia/j...,94.0,123906.0,132354.0,8448.0,2017.0,10.0,31.0,0.0,0.0
2,U.S. President Donald Trump fired the federal ...,http://www.reuters.com/article/us-usa-trump-im...,81.0,121533.0,158776.0,37243.0,2017.0,1.0,31.0,0.0,0.0
3,James Comey terminated as Director of FBI,http://abcn.ws/2qPcnnU,88.0,110063.0,127441.0,17378.0,2017.0,5.0,9.0,0.0,0.0
4,"American Icon and Playboy Founder, Hugh M. Hef...",http://www.prnewswire.com/news-releases/americ...,84.0,108121.0,133561.0,25440.0,2017.0,9.0,28.0,0.0,0.0
5,Pet Store Bill Passes California Senate 38 to ...,https://lbpost.com/life/pets/2000011459-pet-st...,92.0,106866.0,117043.0,10177.0,2017.0,9.0,13.0,0.0,0.0
6,CNN banned from White House press briefing,http://money.cnn.com/2017/02/24/media/cnn-bloc...,84.0,102360.0,126444.0,24084.0,2017.0,2.0,24.0,0.0,0.0
7,Boy who suffered up to 100 seizures a day has ...,https://www.independent.co.uk/news/uk/home-new...,83.0,101359.0,127466.0,26107.0,2017.0,9.0,7.0,0.0,0.0
8,YouTube star Daddyofive loses custody of two c...,http://www.independent.co.uk/life-style/gadget...,89.0,100902.0,115131.0,14229.0,2017.0,5.0,2.0,0.0,0.0
9,Carrie Fisher Dies at 60,http://people.com/movies/carrie-fisher-dies/,81.0,99275.0,129697.0,30422.0,2016.0,12.0,27.0,0.0,0.0


In [5]:
# daily hot/top-by-24hr data collector



In [9]:
######################
#                    #
#   keyword search   #
#                    #
######################

import newspaper
from newspaper import Article
import operator
import rake as rake
rake_object = rake.Rake("SmartStoplist.txt", 1, 2, 1)

In [12]:
words = {}

for i in range(len(threads_df)):
    #top10news_df.iloc[i]['url']
    myArticle = Article(threads_df.iloc[i]['URL'])
    myArticle.download()
    myArticle.parse()
    myArticle.nlp()
    for keyword in myArticle.keywords:
        if keyword not in words:
            words[keyword] = [keyword, 1, threads_df.iloc[i]['# of Upvotes']]
        else:
            words[keyword][1] += 1
            words[keyword][2] += threads_df.iloc[i]['# of Upvotes']
    #RAKE STUFF HERE
    for wordPair in rake_object.run(threads_df.iloc[i]['Title']):
        currentWord = wordPair[0]
        #print(currentWord)
        if currentWord not in words:
            words[currentWord] = [currentWord, 1, threads_df.iloc[i]['# of Upvotes']]
        else:
            words[currentWord][1] += 1
            words[currentWord][2] += threads_df.iloc[i]['# of Upvotes']
keywords_df = pd.DataFrame(words, index=['Keyword','Occurences','Upvotes'])
keywords_df = keywords_df.transpose().set_index('Keyword')
keywords_df

,Occurences,Upvotes
Keyword,,
100,1,127466
100 seizures,1,127466
2017,1,132354
300,1,127466
300 days,1,127466
38,1,117043
41,1,152666
485,1,117043
60,1,129697


In [66]:
keywords = []
for i in range(0, len(threads_df['Self Post?'])):
    if (threads_df['Self Post?'][i] == 0.0):
        a = Article(threads_df['URL'][i], language = 'en')
        a.download()
        a.parse()
        a.text[:150]
        a.nlp()
        for j in a.keywords:
            keywords.append(j)
            
x = dict([x,keywords.count(x)] for x in set(keywords))
sorted_x = sorted(x.items(), key=operator.itemgetter(1))

In [67]:
print(sorted_x) #list of keywords sorted from least to greatest
search = input('enter keyword here: ')
if (search in x):
    print(search, 'appears', x[search], 'times')
else:
    print('that keyword did not appear')

[('day', 1), ('300', 1), ('loses', 1), ('mr', 1), ('bennington', 1), ('security', 1), ('minister', 1), ('suffered', 1), ('caldwell', 1), ('magazine', 1), ('cnn', 1), ('parents', 1), ('remember', 1), ('seizure', 1), ('38', 1), ('american', 1), ('featured', 1), ('soon', 1), ('office', 1), ('information', 1), ('success', 1), ('suicide', 1), ('inducted', 1), ('vote', 1), ('seizures', 1), ('million', 1), ('immigration', 1), ('media', 1), ('prank', 1), ('award', 1), ('revealed', 1), ('true', 1), ('justice', 1), ('way', 1), ('died', 1), ('classified', 1), ('senate', 1), ('ambassador', 1), ('order', 1), ('passes', 1), ('comey', 1), ('film', 1), ('485', 1), ('department', 1), ('later', 1), ('children', 1), ('passed', 1), ('carrie', 1), ('band', 1), ('enterprises', 1), ('chester', 1), ('attorney', 1), ('animals', 1), ('boy', 1), ('wars', 1), ('blocks', 1), ('dogs', 1), ('singer', 1), ('store', 1), ('trumps', 1), ('uk', 1), ('video', 1), ('briefing', 1), ('fbi', 1), ('shelters', 1), ('prescribed'